In [48]:
import pandas as pd
from pandas.plotting import scatter_matrix


In [49]:
dataset = pd.read_csv("merged_matches.csv", index_col=0)

In [50]:
columns_for_ML = {
    'goals_diff': (dataset['h_goals']-dataset['a_goals']),
    'total_shots_diff': (dataset['h_total_shots']-dataset['a_total_shots']),
    'shots_on_target_diff': (dataset['h_shots_on_target']-dataset['a_shots_on_target']),
    'goals_on_penalty_diff': (dataset['h_goals_on_penalty']-dataset['a_goals_on_penalty']),
    'total_penalties_diff': (dataset['h_total_penalties']-dataset['a_total_penalties']),
    'completed_passings_diff': (dataset['h_completed_passings']-dataset['a_completed_passings']),
    'total_passings_diff': (dataset['h_total_passings']-dataset['a_total_passings']),
    'corners_diff': (dataset['h_corners']-dataset['a_corners']), 
    'fouls_diff': (dataset['h_fouls']-dataset['a_fouls']), 
    'yellow_cards_diff': (dataset['h_yellow_cards']-dataset['a_yellow_cards']), 
    'red_cards_diff': (dataset['h_red_cards']-dataset['a_red_cards']), 
    'percentage_possession_diff': (dataset['h_percentage_possession']-dataset['a_percentage_possession']),
    #'risultato': dataset[['Risultato']].replace(['V','P','N'],[0,1,2])
}

In [51]:
results = dataset[['Risultato']].replace(['V','P','N'],[0,1,2])
dataset = dataset.drop(['Risultato'], axis=1)

for key, value in columns_for_ML.items():
    dataset[key] = value

dataset.head()

,h_team,h_goals,a_team,a_goals,h_total_shots,h_shots_on_target,h_goals_on_penalty,h_total_penalties,h_completed_passings,h_total_passings,...,shots_on_target_diff,goals_on_penalty_diff,total_penalties_diff,completed_passings_diff,total_passings_diff,corners_diff,fouls_diff,yellow_cards_diff,red_cards_diff,percentage_possession_diff
0,Hellas Verona,2,Sassuolo,3,14.0,3.0,1,1,282.0,361.0,...,-4.0,1,1,-199.0,-179.0,3.0,-5.0,1,1,-20.0
0,Inter,4,Genoa,0,17.0,8.0,0,0,563.0,630.0,...,4.0,0,0,198.0,200.0,6.0,11.0,-1,0,18.0
0,Torino,1,Atalanta,2,21.0,7.0,0,0,304.0,413.0,...,5.0,0,0,-24.0,-49.0,4.0,3.0,0,0,-6.0
0,Empoli,1,Lazio,3,17.0,4.0,0,0,410.0,496.0,...,1.0,-1,-1,-107.0,-93.0,2.0,5.0,1,0,-8.0
0,Udinese,2,Juventus,2,11.0,5.0,1,1,363.0,433.0,...,2.0,1,1,-117.0,-118.0,0.0,-2.0,-2,0,-12.0


In [52]:
teams = dataset['h_team'].drop_duplicates()
teams.values

home_teams = [name+'_h' for name in teams.values]
away_teams = [name+'_a' for name in teams.values]

In [53]:
dataset_ML = dataset[[k for k in columns_for_ML.keys()]]

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train,X_test,y_train,y_test = train_test_split(dataset_ML, results, test_size=0.2, stratify = results)

In [56]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

rf.fit(X_test, y_test)


<ipython-input-56-b3e309f88fbe>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_test, y_test)


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [57]:
preds = rf.predict(X_test)

In [58]:
from sklearn.metrics import accuracy_score
error = accuracy_score(y_test, preds)
error

1.0

# Other procedure

In [59]:
n_matches, features = dataset.shape
home_wins = len(dataset[dataset["Risultato"]=='V'])
home_losts = len(dataset[dataset["Risultato"]=='P'])
draws = len(dataset[dataset["Risultato"]=='N'])

rate_home_wins = home_wins * 100 / n_matches
rate_home_losts = home_losts * 100 / n_matches
rate_draws = draws * 100 / n_matches

print("Total number of matches: {}".format(n_matches))
print("Number of features: {}".format(features))
print("Number of matches won by home team: {}".format(home_wins))
print("Win rate of home team: {}".format(rate_home_wins))

print("Number of matches lost by home team: {}".format(home_losts))
print("Losts rate of home team: {}".format(rate_home_losts))

print("Number of draw matches: {}".format(draws))
print("Draws rate: {}".format(rate_draws))

KeyError: 'Risultato'

In [ ]:
columns_needed = ['h_goals', 'a_goals', 
        'h_total_shots', 'h_shots_on_target', 'a_total_shots', 'a_shots_on_target', 
        'h_goals_on_penalty', 'h_total_penalties', 'a_goals_on_penalty', 'a_total_penalties', 
        'h_completed_passings', 'h_total_passings', 'a_completed_passings', 'a_total_passings',
        'h_corners', 'a_corners', 
        'h_fouls', 'h_yellow_cards', 'h_red_cards', 'a_yellow_cards', 'a_red_cards', 'a_fouls', 
        'h_percentage_possession', 'a_percentage_possession']

scatter_matrix(dataset[columns_needed], figsize=(30,30))


In [ ]:
output = pd.DataFrame(index = dataset.index)
for col, col_data in dataset.iteritems():
    if col_data.dtype == object:
        col_data = pd.get_dummies(col_data, prefix = col)

    output = output.join(col_data)

output

: 

: 

In [ ]:
dataset = dataset.dropna()
dataset.shape

(380, 27)